# Outils-corpus 4
# Graphes

## 1. Implémentation Python

![graph](http://clement-plancq.github.io/outils-corpus/files/graphe.png)

Ce graphe peut être implémenté à l'aide d'un dictionnaire python.  
En clé les nœuds, en valeur les destinations des arêtes.  
Les arêtes seront des tuples `('A', 'B')` par exemple.

Vous trouverez de l'information de première main sur ces deux sites dont je me suis inspiré :
  - [https://www.python.org/doc/essays/graphs/](https://www.python.org/doc/essays/graphs/)
  - [https://www.python-course.eu/graphs_python.php](https://www.python-course.eu/graphs_python.php)

In [17]:
my_graph = {
    'A': ['C', 'B', 'E'], # le sommet 'A' est relié aux sommets 'B', 'C', 'E'
    'B': ['F'],
    'C': ['G','H'],
    'D': ['H'],
    'E': ['J'],
    'F': ['I'],
    'G': ['C'],
    'H': ['D','J'],
    'I': ['J'],
    'J': ['E', 'J'],
    }

In [2]:
my_graph

{'A': ['C', 'B', 'E'],
 'B': ['F'],
 'C': ['G', 'H'],
 'D': ['H'],
 'E': ['J'],
 'F': ['I'],
 'G': ['C'],
 'H': ['D', 'J'],
 'I': ['J'],
 'J': ['E', 'J']}

Pour manipuler un graphe il faut pouvoir ajouter un nœud et une arête

In [3]:
def add_node(graph, node):
    if node not in graph:
        graph[node] = []

In [4]:
def add_edge(graph, edge):
    node1, node2 = tuple(edge)
    if node1 in graph:
        graph[node1].append(node2)
    else:
        graph[node1] = [node2]

In [16]:
# hop j'ajoute un nœud
#add_node(my_graph, 'K')
# hop j'ajoute une arête
add_edge(my_graph, ('K', 'J'))
my_graph

{'A': ['C', 'B', 'E'],
 'B': ['F'],
 'C': ['G', 'H'],
 'D': ['H'],
 'E': ['J'],
 'F': ['I'],
 'G': ['C'],
 'H': ['D', 'J'],
 'I': ['J'],
 'J': ['E', 'J'],
 'K': ['J']}

Il faut également pouvoir obtenir facilement la listes de nœuds et des arêtes.  
Je fais les nœuds, vous faîtes les arêtes :

In [20]:
def get_nodes(graph):
    """
    retourne les nœuds d'un graphe 
    Args:
        graph (dict): le graphe
    Returns:
        list of str
    """
    return list(graph.keys())

In [21]:
def get_edges(graph):
    """
    retourne les arêtes d'un graphe 
    Args:
        graph (dict): le graphe
    Returns:
        list of tuples
    """
    edges = []
    for node in graph:
        for edge in graph[node]:
            edges.append((node, edge))
    return edges

In [23]:
get_nodes(my_graph)
get_edges(my_graph)

[('A', 'C'),
 ('A', 'B'),
 ('A', 'E'),
 ('B', 'F'),
 ('C', 'G'),
 ('C', 'H'),
 ('D', 'H'),
 ('E', 'J'),
 ('F', 'I'),
 ('G', 'C'),
 ('H', 'D'),
 ('H', 'J'),
 ('I', 'J'),
 ('J', 'E'),
 ('J', 'J')]

Et enfin il faut une fonction pour trouver les chemins entre deux nœuds

In [24]:
def find_path(graph, start, end, path=None):
    if path == None:
        path = []
    path = path + [start]
    if start == end:
        return path
    if start not in graph:
        return None
    for node in graph[start]:
        print(start, node)
        if node not in path:
            extended_path = find_path(graph, node, end, path)
            if extended_path: 
                return extended_path
    return None

In [29]:
find_path(my_graph, 'A', 'J')

A C
C G
G C
C H
H D
D H
H J


['A', 'C', 'H', 'J']

In [26]:
def find_all_paths(graph, start, end, path=[]):
    if path == None:
        path = []
    path = path + [start]
    if start == end:
        return [path]
    if start not in graph:
        return None
    paths = []
    for node in graph[start]:
        if node not in path:
            newpaths = find_all_paths(graph, node, end, path)
            for newpath in newpaths:
                paths.append(newpath)
    return paths

In [28]:
find_all_paths(my_graph, 'A', 'J')

[['A', 'C', 'H', 'J'], ['A', 'B', 'F', 'I', 'J'], ['A', 'E', 'J']]

In [31]:
from collections import deque
def find_shortest_path(graph, start, end):
        dist = {start: [start]}
        q = deque(start)
        while len(q):
            at = q.popleft()
            for next in graph[at]:
                if next not in dist:
                    dist[next] = dist[at] + [next]
                    q.append(next)
        return dist.get(end)

In [32]:
find_shortest_path(my_graph, 'A', 'J')

['A', 'E', 'J']

## 2. Graphes de données langagières – Grew

Ce qui nous intéresse c'est implémenter sous forme de graphe une donnée comme : ![graph](http://clement-plancq.github.io/outils-corpus/files/sequoia-spacy.svg)

Ce qui suit est inspiré de la [doc de Grew](https://grew.fr/usage/python/)  
On a toujours un dictionnaire avec en clé l'identifiant du mot et en valeur un tuple contenant la forme et la liste des dépendants du mot.

In [33]:
g = dict()
g['W1'] = ('Le', [])
g['W2'] = ('conseil', [])
g['W3'] = ('municipal', [])
g['W4'] = ('donne', [('nsubj', 'W2'), ('obj', 'W6')])
g['W5'] = ('son', [])
g['W6'] = ('accord', [('det', 'W5')])
g['W2'][1].append(('det', 'W1'))
g['W2'][1].append(('amod', 'W3'))

In [39]:
g['W9'] = ('procédure', [])
g['W4'][1].append(('obl', 'W9'))
g['W4']

('donne',
 [('nsubj', 'W2'),
  ('obj', 'W6'),
  ('obl', 'W9'),
  ('obl', 'W9'),
  ('obl', 'W9'),
  ('obl', 'W9')])

On peut ajouter des traits au nœud plutôt que la forme seule.

In [44]:
g = dict()  
g['W1'] = ({'form': 'Le', 'upos': 'DET'}, [])
g['W2'] = ({'form': 'conseil', 'upos': 'NOUN'}, [])
g['W3'] = ({'form': 'municipal', 'upos': 'ADJ'}, [])
g['W4'] = ({'form': 'donne', 'upos': 'VERB', 'lemma':'donner'}, [('nsubj', 'W2'), ('obj', 'W6')])
g['W5'] = ({'form': 'son', 'upos': 'DET'}, [])
g['W6'] = ({'form': 'accord', 'upos': 'NOUN'}, [('det', 'W5')])
g['W2'][1].append(('det', 'W1'))
g['W2'][1].append(('amod', 'W3'))

g['W4'][1]

[('nsubj', 'W2'), ('obj', 'W6')]

Le module Grew permet d'écrire un graphe dans une syntaxe simplifiée
```python
g = grew.graph('''graph {                                                                   
    W1 [form = "Le", lemma = "le", upos = DET];
    W2 [form = "conseil", lemma = "conseil", upos = NOUN];
    W3 [form = "municipal", lemma = "municipal", upos = ADJ];
    W4 [form = "donne", lemma = "donner", upos = VERB];
    W5 [form = "son", lemma = "son", upos = DET];
    W6 [form = "accord", lemma = "accord", upos = NOUN];
    W2 -[det]->W1;
    W2 -[amod]->W3;
    W4 -[nsubj]->W2;
    W4 -[obj]->W6;
    W6 -[det]->W5;
    }''')
```

Il permet également d'utiliser des patrons de recherche comme .
```python
grew.search('''pattern {
     GOV [lemma = "donner"];
     DEP[lemma = "accord"];
     GOV -[obj]-> DEP }', g)
```

In [ ]:
import grew

g = grew.graph('''graph {                                                                   
    W1 [form = "Le", lemma = "le", upos = DET];
    W2 [form = "conseil", lemma = "conseil", upos = NOUN];
    W3 [form = "municipal", lemma = "municipal", upos = ADJ];
    W4 [form = "donne", lemma = "donner", upos = VERB];
    W5 [form = "son", lemma = "son", upos = DET];
    W6 [form = "accord", lemma = "accord", upos = NOUN];
    W2 -[det]->W1;
    W2 -[amod]->W3;
    W4 -[nsubj]->W2;
    W4 -[obj]->W6;
    W6 -[det]->W5;
    }''')

In [ ]:
g

Grew peut s'utiliser :
  - en ligne de commande sur des fichiers au format CoNLL-U
  - via la [lib python](https://pypi.org/project/Grew/)

Je ne vous demande pas d'installer Grew sur vos machines. Vous utiliserez l'interface web [Grew-match](http://match.grew.fr/).  
Grew est un outil de réécriture de graphes (*Graph rewriting for NLP*), la recherche de patrons n'en est qu'une sous-partie


Aidez de la [documentation sur les motifs](http://grew.fr/pattern/) et du tutoriel de [Grew-match](http://match.grew.fr/?corpus=UD_French-Sequoia@2.7) pour trouver dans le corpus `UD_French-Sequoia@2.7` :
  - les triplets sujet, verbe, objet
  - les phrases avec sujets inversés


## 3. Graphes de données langagières – Spacy


La librairie [spaCy](https://spacy.io/) propose dans sa chaîne de traitement une analyse syntaxique en dépendance.

Le modèle pour le français est appris sur le corpus [Sequoia](http://deep-sequoia.inria.fr/) version UD et [wikiNER](https://figshare.com/articles/Learning_multilingual_named_entity_recognition_from_Wikipedia/5462500)

Nous verrons plus en détail comment utiliser Spacy lors de la prochaine séance.

In [5]:
import spacy
nlp = spacy.load('fr_core_news_md')
doc = nlp("Le conseil municipal donne son accord pour cette procédure.")
for tok in doc:
    print(tok.text, tok.pos_)

Le DET
conseil NOUN
municipal ADJ
donne VERB
son DET
accord NOUN
pour ADP
cette DET
procédure NOUN
. PUNCT


In [6]:
from spacy import displacy
displacy.render(doc, style="dep", jupyter=True, options={'distance':110})

Il existe également un outil issu d'un développement indépendant : [explacy](https://spacy.io/universe/project/explacy)

In [9]:
import explacy
explacy.print_parse_info(nlp, 'Derrière lui, sur le carreau de la rue Rambuteau, on vendait des fruits.')

Dep tree     Token     Dep type Lemma     Part of Sp
──────────── ───────── ──────── ───────── ──────────
         ┌─► Derrière  case     derrière  ADP       
┌───────►└── lui       obl:mod  lui       PRON      
│┌─────────► ,         punct    ,         PUNCT     
││      ┌──► sur       case     sur       ADP       
││      │┌─► le        det      le        DET       
││┌─►┌──┴┴── carreau   obl:mod  carreau   NOUN      
│││  │  ┌──► de        case     de        ADP       
│││  │  │┌─► la        det      le        DET       
│││  └─►└┼── rue       nmod     rue       NOUN      
│││      └─► Rambuteau nmod     Rambuteau PROPN     
│││     ┌──► ,         punct    ,         PUNCT     
│││     │┌─► on        nsubj    on        PRON      
└┴┴──┬┬─┴┴── vendait   ROOT     vendre    VERB      
     ││  ┌─► des       det      un        DET       
     │└─►└── fruits    obj      fruit     NOUN      
     └─────► .         punct    .         PUNCT     


Pour ce qui nous intéresse aujourd'hui vous devez lire la [doc sur l'analyse en dépendance](https://spacy.io/usage/linguistic-features#dependency-parse)

Pour chaque `token` on peut accéder à:
  - sa tête `token.head`
  - le label de la relation `token.dep_`
  - les tokens régis `token.children` (seulement à gauche `token.lefts`, à droite `token.rights`, la séquence complète `token.subtree`)
  - la chaîne de recteurs `token.ancestors`

In [7]:
root = [token for token in doc if token.head == token][0]
subjects = [tok for tok in root.lefts if tok.dep_ == "nsubj"]
subject = subjects[0]

In [8]:
for descendant in subject.subtree:
    print(descendant.text)

Le
conseil
municipal


In [ ]:
root = [token for token in doc if token.head == token][0]
objs = [tok for tok in root.rights if tok.dep_ == "obj"]
obj = objs[0]

In [ ]:
for descendant in obj.subtree:
    print(descendant.text)

Pour la séance prochaine,  extrayez les triplets (sujet, verbe, objet) des phrases suivantes et commentez les éventuelles erreurs ou manques.

    « Les enfants n'aiment pas trop les asperges. »
    « Les Français réclament moins d'impôts. »
    « Les acacias donnent un miel ambré, limpide et fluide. »
    « L'équipe fait porter le chapeau à l'arbitrage. »
    « Des nuées de milliards d'insectes, venus de la péninsule Arabique, s'abattent sur la Corne de l'Afrique et dévorent les cultures, mettant en péril la sécurité alimentaire de la région. »


In [ ]:
doc = nlp("Des nuées de milliards d'insectes, venus de la péninsule Arabique, s'abattent sur la Corne de l'Afrique et dévorent les cultures, mettant en péril la sécurité alimentaire de la région.")

In [ ]:
displacy.render(doc, style="dep", jupyter=True, options={'distance':110})

Depuis la v3, Spacy a ajouté un *Dependancy Matcher* qui permet de faire de l'extraction de patrons syntaxiques. Il est maintenant possible de faire porter des requêtes sur l'arbre syntaxique et non plus seulement sur la séquence des tokens.  
Ce dispositif utilise [Semgrex](https://nlp.stanford.edu/nlp/javadoc/javanlp/edu/stanford/nlp/semgraph/semgrex/SemgrexPattern.html), la syntaxe utilisée dans Tgrep et Tregex, les outils de requête sur Treebank de Stanford.

Voir la [documentation](https://spacy.io/usage/rule-based-matching#dependencymatcher)

In [11]:
ventre_short = ""
with open('files/Le_Ventre_de_Paris-short.txt') as input_f:
    ventre_short = input_f.read()
doc = nlp(ventre_short)

In [12]:
from spacy.matcher import DependencyMatcher

matcher = DependencyMatcher(nlp.vocab)
pattern = [
  {
    "RIGHT_ID": "vendre",    
    "RIGHT_ATTRS": {"LEMMA": "vendre"}
  }
]
matcher.add("VENDRE", [pattern])
matches = matcher(doc)
for m_id, t_ids in matches:
    for t_id in t_ids:
        print(doc[t_id])

vend
vendant
vendait
vendait
vendaient
vendaient
vend
vendu
vendu
vendre
vendait
vendu
vendais
vendu
vendrait


In [13]:
from spacy.matcher import DependencyMatcher

matcher = DependencyMatcher(nlp.vocab)
pattern = [
    {
        "RIGHT_ID": "vendre",    
        "RIGHT_ATTRS": {"LEMMA": {"IN": ["vendre", "acheter"]}}
    },
    {
        "LEFT_ID": "vendre",
        "REL_OP": ">",
        "RIGHT_ID": "sujet",
        "RIGHT_ATTRS": {"DEP": "nsubj"},  
    },
    {
        "LEFT_ID": "vendre",
        "REL_OP": ">",
        "RIGHT_ID": "objet",
        "RIGHT_ATTRS": {"DEP": {"IN": ["obj", "iobj", "obl"]}},  
    }
]
matcher.add("VENDRE", [pattern])
matches = matcher(doc)
for m_id, t_ids in matches:
    print("verbe, sujet, objet : ", " -> ".join([doc[t_id].text for t_id in t_ids]))
    print("objet complet : ", " ".join([t.text for t in doc[t_ids[2]].subtree]))
    print("Phrase compléte : ", doc[t_ids[0]].sent)
    print()

verbe, sujet, objet :  acheta -> il -> derniers
objet complet :  ses deux derniers sous de pain
Phrase compléte :  Mais, à Vernon, il acheta ses deux derniers sous de pain.

verbe, sujet, objet :  achetait -> elle -> qu’
objet complet :  qu’
Phrase compléte :  J’étais gamine, qu’elle achetait déjà ses navets à mon père.

verbe, sujet, objet :  achetait -> elle -> navets
objet complet :  ses navets à mon père
Phrase compléte :  J’étais gamine, qu’elle achetait déjà ses navets à mon père.

verbe, sujet, objet :  vendait -> on -> fruits
objet complet :  des fruits
Phrase compléte :  

Derrière lui, sur le carreau de la rue Rambuteau, on vendait des fruits.

verbe, sujet, objet :  vendaient -> qui -> bottes
objet complet :  des bottes de fougère et des paquets de feuilles de vigne , bien réguliers , attachés par quarterons
Phrase compléte :  Ils s’arrêtèrent curieusement devant des femmes qui vendaient des bottes de fougère et des paquets de feuilles de vigne, bien réguliers, attachés par 